## Cloning the Repository

In [ ]:
!git clone --recursive https://github.com/v-iashin/BMT.git
%cd BMT/

Cloning into 'BMT'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 182 (delta 42), reused 40 (delta 37), pack-reused 123
Receiving objects: 100% (182/182), 12.86 MiB | 18.93 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Submodule 'submodules/pycocoevalcap' (https://github.com/salaniz/pycocoevalcap.git) registered for path 'submodules/pycocoevalcap'
Submodule 'submodules/video_features' (https://github.com/v-iashin/video_features.git) registered for path 'submodules/video_features'
Cloning into '/content/BMT/BMT/submodules/pycocoevalcap'...
remote: Enumerating objects: 821, done.        
remote: Counting objects: 100% (24/24), done.        
remote: Compressing objects: 100% (20/20), done.        
remote: Total 821 (delta 5), reused 19 (delta 4), pack-reused 797        
Receiving objects: 100% (821/821), 130.06 MiB | 26.82 MiB/s, done.
Resolving deltas: 100% (424/424), done.
Cloni

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh -q --show-progress
!bash ./Miniconda3-py37_23.1.0-1-Linux-x86_64.sh -b -f -p /usr/local

Miniconda3-py37_23. 100%[===================>]  86.46M   219MB/s    in 0.4s    
PREFIX=/usr/local
Unpacking payload ...
Traceback (most recent call last):
  File "entry_point.py", line 69, in <module>

[50065] Failed to execute script 'entry_point' due to unhandled exception!
Process ForkProcess-2:
Exception ignored in: <module 'threading' from '/usr/local/install_tmp/_MEI4zCfMQ/threading.pyc'>
Exception ignored in sys.unraisablehook: <built-in function unraisablehook>
Traceback (most recent call last):
KeyboardInterrupt
  File "multiprocessing/process.py", line 315, in _bootstrap
  File "multiprocessing/process.py", line 108, in run
  File "concurrent/futures/process.py", line 240, in _process_worker
  File "multiprocessing/queues.py", line 103, in get
  File "multiprocessing/connection.py", line 221, in recv_bytes
  File "multiprocessing/connection.py", line 419, in _recv_bytes
  File "multiprocessing/connection.py", line 384, in _recv
KeyboardInterrupt
^C


In [ ]:
#@title
# fixing nasty colab environment
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install cryptography==38.0.4
!pip install pyopenssl --upgrade

In [ ]:
import os
from pathlib import Path
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
from sample.single_video_prediction import get_video_duration

ModuleNotFoundError: ignored

In [ ]:
# feature extraction
!export PIP_DEFAULT_TIMEOUT=100
!conda env create -f ./submodules/video_features/conda_env_i3d.yml
!conda env create -f ./submodules/video_features/conda_env_vggish.yml
# captioning model
!conda env create -f ./conda_env.yml
# spacy language model
!/usr/local/envs/bmt/bin/python -m spacy download en


EnvironmentFileNotFound: '/content/submodules/video_features/conda_env_i3d.yml' file not found


EnvironmentFileNotFound: '/content/submodules/video_features/conda_env_vggish.yml' file not found


EnvironmentFileNotFound: '/content/conda_env.yml' file not found

Traceback (most recent call last):
  File "/usr/local/envs/bmt/lib/python3.7/runpy.py", line 183, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/local/envs/bmt/lib/python3.7/runpy.py", line 142, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/local/envs/bmt/lib/python3.7/runpy.py", line 109, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/envs/bmt/lib/python3.7/site-packages/spacy/__init__.py", line 4, in <module>
    from .cli.info import info as cli_info
  File "/usr/local/envs/bmt/lib/python3.7/site-packages/spacy/cli/__init__.py", line 1, in <module>
    from .download import download
  File "/usr/local/env

Downloading the GloVe embeddings. `torchtext` could do it for us but it uses a slow server.
Therefore, we store it on our premises.

In [ ]:
!wget https://a3s.fi/swift/v1/AUTH_a235c0f452d648828f745589cde1219a/bmt/glove.840B.300d.zip -q --show-progress
!wget https://a3s.fi/swift/v1/AUTH_a235c0f452d648828f745589cde1219a/bmt/best_cap_model.pt -q --show-progress
!wget https://a3s.fi/swift/v1/AUTH_a235c0f452d648828f745589cde1219a/bmt/best_prop_model.pt -q --show-progress
!wget https://storage.googleapis.com/audioset/vggish_model.ckpt -q --show-progress

!mkdir .vector_cache
!mv glove.840B.300d.zip ./.vector_cache/
!mv best_cap_model.pt ./sample/
!mv best_prop_model.pt ./sample/
!mv vggish_model.ckpt ./submodules/video_features/models/vggish/checkpoints/

glove.840B.300d.zip 100%[===================>]   2.03G  17.5MB/s    in 2m 3s   
best_cap_model.pt   100%[===================>] 589.61M  18.1MB/s    in 36s     
best_prop_model.pt  100%[===================>]   2.50G  16.5MB/s    in 2m 29s  
vggish_model.ckpt   100%[===================>] 277.62M  92.3MB/s    in 3.0s    


Upload your video to Google Colab and select it in the `MY_VIDEO_PATH` variable.
The script will prepare all necessary file paths and calculate the video duration for you

In [ ]:
# upload a video
MY_VIDEO_PATH = '/content/BMT/sample/two_man_greeting.mp4'

# Preparing the paths
VIDEO_DURATION = get_video_duration(MY_VIDEO_PATH)

FEATURES_CACHE_PATH = '/content/BMT/tmp/'
FEATURES_PATH_STUB = os.path.join(FEATURES_CACHE_PATH, Path(MY_VIDEO_PATH).stem)
FEATURE_PATH_VGGISH = f'{FEATURES_PATH_STUB}_vggish.npy'
FEATURE_PATH_RGB = f'{FEATURES_PATH_STUB}_rgb.npy'
FEATURE_PATH_FLOW = f'{FEATURES_PATH_STUB}_flow.npy'

PROPOSAL_CKPT = '/content/BMT/sample/best_prop_model.pt'
CAPTIONING_CKPT = '/content/BMT/sample/best_cap_model.pt'

Video Duration: 3.512


The script will extract audio and visual features from the video

In [ ]:
# Extract I3D features (visual)
!cd ./submodules/video_features && /usr/local/envs/i3d/bin/python main.py \
    --feature_type i3d \
    --on_extraction save_numpy \
    --device_ids 0 \
    --extraction_fps 25 \
    --video_paths $MY_VIDEO_PATH \
    --output_path $FEATURES_CACHE_PATH

# Extract VGGish features (audio)
!cd ./submodules/video_features && /usr/local/envs/vggish/bin/python main.py \
    --feature_type vggish \
    --on_extraction save_numpy \
    --device_ids 0 \
    --video_paths $MY_VIDEO_PATH \
    --output_path $FEATURES_CACHE_PATH

Saving features to /content/BMT/tmp/
100% 1/1 [00:30<00:00, 30.62s/it]
Saving features to /content/BMT/tmp/

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

100% 1/1 [00:11<00:00, 11.20s/it]


Run dense video captioning

In [ ]:
# captioning parameters
MAX_PROP_PER_VIDEO = 100
NMS_TIOU_THRESHOLD = 0.4

# Running single video prediction
!/usr/local/envs/bmt/bin/python ./sample/single_video_prediction.py \
    --prop_generator_model_path $PROPOSAL_CKPT \
    --pretrained_cap_model_path $CAPTIONING_CKPT \
    --vggish_features_path $FEATURE_PATH_VGGISH \
    --rgb_features_path $FEATURE_PATH_RGB \
    --flow_features_path $FEATURE_PATH_FLOW \
    --duration_in_secs $VIDEO_DURATION \
    --device_id 0 \
    --max_prop_per_vid $MAX_PROP_PER_VIDEO \
    --nms_tiou_thresh $NMS_TIOU_THRESHOLD

Contructing caption_iterator for "train" phase
100% 2196016/2196017 [05:09<00:00, 7089.58it/s]
Using vanilla Generator
initialization: xavier
Glove emb of the same size as d_model_caps
Pretrained caption path: 
 /content/BMT/sample/best_cap_model.pt
[{'start': 0.0, 'end': 1.7, 'sentence': 'A man is seen standing on a table with a woman standing on a woman in a room'}, {'start': 0.1, 'end': 0.6, 'sentence': 'A man is seen standing on a table with a woman standing on a woman in a room'}, {'start': 2.5, 'end': 3.5, 'sentence': 'A man in a white shirt is standing on a table'}, {'start': 0.6, 'end': 3.2, 'sentence': 'A man is seen standing on a table with a woman standing on a woman in a room'}, {'start': 3.2, 'end': 3.5, 'sentence': 'A man in a white shirt is standing on a table'}, {'start': 0.4, 'end': 0.7, 'sentence': 'A man is seen standing on a table with a woman standing on a woman in a room'}, {'start': 0.8, 'end': 1.1, 'sentence': 'A man is seen standing on a table with a woman stan